<a href="https://colab.research.google.com/github/BhuvanKalyanGV/Web_scapping/blob/main/eex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver



In [ ]:
!pip install webdriver_manager

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Fetch gas types and values using BeautifulSoup
url = 'https://www.eex.com/en/market-data/natural-gas/futures'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
gas_types = []
gas_values = soup.find('select', {'class': 'selectpicker'}).find_all('option')
for gas in gas_values:
    gas_types.append({
        "name": gas.text,
        "value": gas.get('value')
    })

# Print gas types and values
print("Gas types:")
for gas in gas_types:
    print(f"{gas['name']} ({gas['value']})")

# Get user input for gas type and date
gas_value = input("Enter the value of the gas type: ")
date = input("Enter the date (yyyy-mm-dd): ")

# Format URL with selected gas type
url = f'https://www.eex.com/en/market-data/natural-gas/futures#%7B%22snippetpicker%22%3A%22{gas_value}%22%7D'

# Use Selenium to input date and search
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get(url)

# Input date
date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input#date")))
date_input.clear()
date_input.send_keys(date)
date_input.send_keys(Keys.RETURN)
time.sleep(1)

# Click on category
category_buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.mv-button-group.mv-stack-block.mv-hyperlink-group div.mv-button-base.mv-hyperlink-button.mv-item-selected")))
for button in category_buttons:
    button.click()
    time.sleep(1)

# Scrape table
table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.mv-quote")))
headers = [th.text for th in table.find_elements_by_css_selector("thead tr th")]
rows = []
for tr in table.find_elements_by_css_selector("tbody tr"):
    rows.append([td.text for td in tr.find_elements_by_css_selector("td")])
df = pd.DataFrame(rows, columns=headers)

print(df)

df.to_excel('output.xlsx', index=False)

driver.quit()